# [Day 11](https://adventofcode.com/2022/day/11)

In [1]:
from typing import List, Callable, Dict, Any, Optional
import math
import re
from aoc2022.utils import timeit, get_numbers_from_string

def parse_monkey_id(string: str) -> int:
    return get_numbers_from_string(string, 1)[0]

def parste_starting_items(string: str) -> List[int]:
    return get_numbers_from_string(string)

def parse_operation(string: str) -> Callable:
    if string.count('old') == 2:
        return lambda x: x*x
    number = get_numbers_from_string(string, 1)[0]
    if '+' in string:
        return lambda x: x + number
    if '*' in string:
        return lambda x: x * number
    
def parse_test(string: str) -> Callable:
    number = get_numbers_from_string(string, 1)[0]
    return lambda x: x % number == 0, number

def parse_conditions(condition_true: str, condition_false: str) -> Callable:
    true_value = get_numbers_from_string(condition_true, 1)[0]
    false_value = get_numbers_from_string(condition_false, 1)[0]
    return lambda x: true_value if x else false_value

class Monkey:
    monkies: Dict[int, Any] = dict()
    n_round = 0
    def __init__(self, id: int, items: List[int], operation: Callable, test: Callable, condition: Callable):
        self.id = id
        self.items = items
        self.operation = operation
        self.test = test
        self.condition = condition
        self.inspected_items = 0
        Monkey.monkies[self.id] = self
        
    def turn(self, worry_reducer: Callable):
        for item in self.items:
            self.inspected_items += 1
            item = self.operation(item)
            item = worry_reducer(item)
            test = self.test(item)
            throw_to = self.condition(test)
            Monkey.monkies[throw_to].receive_item(item)
        self.items = []
        
    def receive_item(self, item: int):
        self.items.append(item)
    
    @classmethod
    def round(cls, worry_reducer: Callable, verbose: bool = False):
        for monkey in cls.monkies.values():
            monkey.turn(worry_reducer)
        cls.n_round += 1
        if verbose:
            if cls.n_round in list(range(1,21)) + list(range(1000,11000,1000)):
                print(f"== After round {cls.n_round} == ")
                for id, monkey in cls.monkies.items():
                    print(f"Monkey {id}: {monkey.inspected_items}")
                print("\n\n")
            
    @classmethod
    def inspected_per_monkey(cls, top_n):
        top_inspected = [monkey.inspected_items for monkey in cls.monkies.values()]
        return math.prod(sorted(top_inspected)[-top_n:])


def create_start(path: str) -> Dict[int, Monkey]:
    monkies = dict()
    modulas = 1
    with open(path) as f:
        input_ = f.read()
    monkey_inputs = [line.split('\n') for line in input_.split("\n\n")]
    for monkey in monkey_inputs:
        id, starting_items, operation, test, condition_true, condition_false = monkey
        id = parse_monkey_id(id)
        starting_items = parste_starting_items(starting_items)
        operation = parse_operation(operation)
        test, modula = parse_test(test)
        modulas *= modula
        condition = parse_conditions(condition_true, condition_false)
        monkies[id] = Monkey(id, starting_items, operation, test, condition)
    return monkies, lambda x: x % modulas

@timeit(1000)
def part_one(path: str, n_round: int, verbose: bool = False):
    monkies, _ = create_start(path)
    for _ in range(n_round):
        Monkey.round(lambda x: int(x/3), verbose)
    return Monkey.inspected_per_monkey(2)

@timeit(1000)
def part_two(path: str, n_round: int):
    monkies, worry_reducer = create_start(path)
    for _ in range(n_round):
        Monkey.round(worry_reducer)
    return Monkey.inspected_per_monkey(2)
    

    

        

In [2]:
assert part_one('test_input.txt', 20) == 10605
assert part_one('input.txt', 20) == 50616

'part_one()' took on average 0.00017487072944641114 seconds with a stdev of 15.54%. (1000 runs)
'part_one()' took on average 0.0004945342540740966 seconds with a stdev of 5.24%. (1000 runs)


In [3]:
assert part_two('test_input.txt', 10000) == 2713310158
assert part_two('input.txt', 10000) == 11309046332

'part_two()' took on average 0.0598579261302948 seconds with a stdev of 3.44%. (1000 runs)
'part_two()' took on average 0.19460829305648802 seconds with a stdev of 1.05%. (1000 runs)
